###Configs Iniciais

In [30]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler, MinMaxScaler

!pip install sklearn_extensions
from sklearn_extensions.extreme_learning_machines.elm import GenELMClassifier
from sklearn_extensions.extreme_learning_machines.random_layer import RBFRandomLayer, MLPRandomLayer

!pip install quickda
from quickda.explore_data import explore

###Carregando e explorando o dataset, para validar ausência de nulos, variaveis numéricas e categoricas

Relação das variáveis (copiado do kaggle):
<ol>
<li>cap-shape: bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s</li>
<li>cap-surface: fibrous=f, grooves=g, scaly=y, smooth=s</li>
<li>cap-color: brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y</li>
<li>bruises: bruises=t, no=f</li>
<li>odor: almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s</li>
<li>gill-attachment: attached=a, descending=d, free=f, notched=n</li>
<li>gill-spacing: close=c, crowded=w, distant=d</li>
<li>gill-size: broad=b, narrow=n</li>
<li>gill-color: black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u, red=e, white=w, yellow=y</li>
<li>stalk-shape: enlarging=e, tapering=t</li>
<li>stalk-root: bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=?</li>
<li>stalk-surface-above-ring: fibrous=f, scaly=y, silky=k, smooth=s</li>
<li>stalk-surface-below-ring: fibrous=f, scaly=y, silky=k, smooth=s</li>
<li>stalk-color-above-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y</li>
<li>stalk-color-below-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y</li>
<li>veil-type: partial=p, universal=u</li>
<li>veil-color: brown=n, orange=o, white=w, yellow=y</li>
<li>ring-number: none=n, one=o, two=t</li>
<li>ring-type: cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z</li>
<li>spore-print-color: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y</li>
<li>population: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y</li>
<li>habitat: grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d</li>
</ol>

In [31]:
mush_ds = pd.read_csv('./mushrooms.csv') #arquivo upload no collab
explore(mush_ds)

,dtypes,count,null_sum,null_pct,nunique,min,25%,50%,75%,max,mean,median,std,skew
bruises,object,8124,0,0.0,2,f,-,-,-,t,-,-,-,-
cap-color,object,8124,0,0.0,10,b,-,-,-,y,-,-,-,-
cap-shape,object,8124,0,0.0,6,b,-,-,-,x,-,-,-,-
cap-surface,object,8124,0,0.0,4,f,-,-,-,y,-,-,-,-
class,object,8124,0,0.0,2,e,-,-,-,p,-,-,-,-
gill-attachment,object,8124,0,0.0,2,a,-,-,-,f,-,-,-,-
gill-color,object,8124,0,0.0,12,b,-,-,-,y,-,-,-,-
gill-size,object,8124,0,0.0,2,b,-,-,-,n,-,-,-,-
gill-spacing,object,8124,0,0.0,2,c,-,-,-,w,-,-,-,-
habitat,object,8124,0,0.0,7,d,-,-,-,w,-,-,-,-


###Conversão de variáveis para categoricas

In [32]:
mush_ds_categ = pd.get_dummies(mush_ds,drop_first=False)
mush_ds_categ.head()

,class_e,class_p,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_b,cap-color_c,cap-color_e,cap-color_g,cap-color_n,cap-color_p,cap-color_r,cap-color_u,cap-color_w,cap-color_y,bruises_f,bruises_t,odor_a,odor_c,odor_f,odor_l,odor_m,odor_n,odor_p,odor_s,odor_y,gill-attachment_a,gill-attachment_f,gill-spacing_c,gill-spacing_w,gill-size_b,gill-size_n,gill-color_b,...,stalk-color-below-ring_n,stalk-color-below-ring_o,stalk-color-below-ring_p,stalk-color-below-ring_w,stalk-color-below-ring_y,veil-type_p,veil-color_n,veil-color_o,veil-color_w,veil-color_y,ring-number_n,ring-number_o,ring-number_t,ring-type_e,ring-type_f,ring-type_l,ring-type_n,ring-type_p,spore-print-color_b,spore-print-color_h,spore-print-color_k,spore-print-color_n,spore-print-color_o,spore-print-color_r,spore-print-color_u,spore-print-color_w,spore-print-color_y,population_a,population_c,population_n,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,...,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,...,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,...,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,...,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,...,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0


###Separação do dataset em treino e validação

In [33]:
x_full = mush_ds_categ.iloc[:].values
y_full = mush_ds_categ.iloc[:,0].values

In [34]:
x_full.shape

(8124, 119)

###Normalizando

In [37]:
x_full = StandardScaler().fit_transform(x_full)

###Divisão treino/teste

In [39]:
x_train, x_test, y_train, y_test = train_test_split(x_full, y_full, test_size = 0.3)

In [40]:
x_train.shape

(5686, 119)

###Definição do modelo ELM



In [41]:
hiddenLayer = 18
funcoes = ['tanh', 'sine', 'tribas', 'sigmoid', 'hardlim', 'softlim', 'gaussian', 'multiquadric', 'inv_multiquadric'] #relu, lrelu, hardtanh, tanhre <= encontrados nas pesquisa, mas não funcionaram nos testes


In [42]:
#EML baseado em MLP

acc = []
dictionary = {}

for funcao in range(len(funcoes)):
  srhl = MLPRandomLayer(n_hidden=hiddenLayer, activation_func=funcoes[funcao], random_state=0)
  elm_model = GenELMClassifier(hidden_layer=srhl)
  elm_model.fit(x_train, y_train)
  acc.append(elm_model.score(x_test, y_test))

dictionary = {'Função':funcoes,'% acerto':acc}

acc_df = pd.DataFrame(dictionary).sort_values(by='% acerto', ascending=False)
acc_df

,Função,% acerto
3,sigmoid,0.956932
0,tanh,0.948318
5,softlim,0.947908
4,hardlim,0.946678
7,multiquadric,0.712469
8,inv_multiquadric,0.660788
6,gaussian,0.596390
2,tribas,0.564807
1,sine,0.513536


In [43]:
#EML baseado em rede RBF

acc = []
dictionary = {}

for funcao in range(len(funcoes)):
  srhl_rbf = RBFRandomLayer(n_hidden=hiddenLayer, rbf_width=1, random_state=0,activation_func=funcoes[funcao])
  elm_model = GenELMClassifier(hidden_layer=srhl_rbf)
  elm_model.fit(x_train, y_train)
  acc.append(elm_model.score(x_test, y_test))

dictionary = {'Função':funcoes,'% acerto':acc}

acc_df = pd.DataFrame(dictionary).sort_values(by='% acerto', ascending=False)
acc_df

,Função,% acerto
6,gaussian,0.843314
0,tanh,0.814192
1,sine,0.811731
3,sigmoid,0.801477
8,inv_multiquadric,0.794094
7,multiquadric,0.791632
4,hardlim,0.521739
5,softlim,0.521739
2,tribas,0.478261


Por conta da acuracia mais elevada, escolhemos o modelo MLPRandomLayer com ativação sigmoid como o campeão